<a href="https://colab.research.google.com/github/samitha278/VLM-gamma/blob/main/repro_siglip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## SigLip

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Optional, Tuple

### SigLip Config

In [3]:
class SigLipConfig:

    def __init__(self,n_embd=768,n_hidden=3072,n_layer=12,n_head=12,n_channel=3,image_size=224,patch_size=16,ln_eps=1e-6,**kwargs):
        super().__init__()

        #model params
        self.n_embd= n_embd
        self.n_hidden = n_hidden
        self.n_layer = n_layer
        self.n_head = n_head
        self.n_channel = n_channel

        #data params
        self.image_size = image_size
        self.patch_size = patch_size
        self.n_patch = (image_size // patch_size)**2

        self.ln_eps = ln_eps


### SigLip ViT Embeddings

In [4]:
class Embeddings(nn.Module):

    def __init__(self,config: SigLipConfig):
        super().__init__()

        self.config = config

        self.patch_embds = nn.Conv2d(in_channels= config.n_channel, out_channels= config.n_embd, kernel_size=config.patch_size, stride= config.patch_size)

        self.pos_embds = nn.Embedding(config.n_patch,embedding_dim=config.n_embd)
        self.register_buffer('pos_ids',torch.arange(0,config.n_patch, persistent=False)) # persistent =False ; don't save in checkpoint

    def forward(self,x):

        # x : [B,C,H,W]

        patch_embds = self.patch_embds(x)    # [B, n_embd, n_patch**0.5, n_patch**0.5]
        patch_embds.flatten(2)               # [B, n_embd, n_patch]
        patch_embds.transpose(-1,-2)         # [B, n_patch, n_embd]

        pos_embds = self.pos_embds(self.pos_ids)

        embds = patch_embds + pos_embds

        return embds



### SigLip ViT Attention

In [5]:
class Attention(nn.Module):

    def __init__(self, config : SigLipConfig ):
        super().__init__()

        self.config = config
        self.head_size = config.n_embd // config.n_head

        self.W = nn.Linear(config.n_embd,config.n_embd*3)

        self.proj = nn.Linear(config.n_Embd,config.n_embd)

    def forward(self,x):

        B,n_patch,n_embd = x.shape

        qkv = self.W(x)   # [B,n_patch,n_embd*3]

        query, key, value = torch.chunk(qkv,3,-1)  # each : [B,n_patch,n_embd]

        query = query.view(B,n_patch,self.config.n_head,self.head_size).transpose(1,2)
        key   =   key.view(B,n_patch,self.config.n_head,self.head_size).transpose(1,2)
        value = value.view(B,n_patch,self.config.n_head,self.head_size).transpose(1,2) # B,n_head,n_patch,head_size

        weights = (query @ key.transpose(-1,-2))/(self.head_size**0.5)  # B,n_head,n_patch,n_patch

        out = weights @ value         # B,n_head,n_patch,head_size

        out = out.transpose(1,2).contiguous().view(B,self.config.n_patch,self.config.n_embd)
        out = self.proj(out)

        return out


In [6]:
class wrongAttention(nn.Module):

    def __init__(self, config : SigLipConfig):
        super().__init__()

        self.config = config

        self.W = nn.Linear(config.n_embd,config.n_embd*3)

        self.proj = nn.Linear(config.n_embd,config.n_embd)


    def forward(self,x):

        # x : [B,n_patch,n_embd]

        qkv = self.W(x)    # [B,n_patch,n_embd*3]

        query,key,value = torch.chunk(qkv,3,-1)  # each : [B,n_patch,n_embd]

        weights = query @ key.transpose(-1,-2)          # [B,n_patch,n_patch]
        weights = weights / (self.config.n_embd//self.config.n_head)**0.5

        out = weights @ value        # [B,n_patch,n_embd]

        out = self.proj(out)
        return out

### SigLip ViT MLP

In [7]:
class MLP(nn.Module):

    def __init__(self,config:SigLipConfig):
        super().__init__()

        self.config = config

        self.layer =  nn.Linear(config.n_embd,config.n_hidden)
        self.gelu = nn.GELU
        self.proj = nn.Linear(config.n_hidden,config.n_embd)

    def forward(self,x):

        out = self.layer(x)
        out = self.gelu(out)
        out = self.proj(out)

        return out

### SigLip ViT Encoder Block

In [8]:
class Block(nn.Module):

    def __init__(self,config:SigLipConfig):
        super().__init__()

        self.config = config

        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = Attention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp = MLP(config)


    def forward(self,x):

        # x : [B,n_patch,n_embd]

        out = x + self.attn(self.ln_1(x))
        out = out + self.mlp(self.ln_2(x))

        return out

### SigLip ViT

In [9]:
class SigLipViT(nn.Module):

    def __init__(self, config: SigLipConfig):
        super().__init__()

        self.config = config

        self.embeddings = Embeddings(config)
        self.encoder = nn.ModuleList([Block(config) for _ in config.n_layer])
        self.ln = nn.LayerNorm(config.n_embd,eps=config.ln_eps)


    def forward(self,x):

        # x : [B,C,H,W]

        embds = self.embeddings(x)

        out = embds
        for block in self.encoder:
            out = block(out)

        out = self.ln(out)

        return out